# Imports

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.tree
#---#
import warnings
warnings.filterwarnings('ignore')
#---#
import matplotlib.animation
import IPython

# Data

In [71]:
np.random.seed(1)
temp = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/temp.csv').iloc[:,3].to_numpy()[:80]
temp.sort()
eps = np.random.randn(80)*3 # 오차
icecream_sales = 20 + temp * 2.5 #+ eps 
df_train = pd.DataFrame({'temp':temp,'sales':icecream_sales})
df_train

,temp,sales
0,-4.1,9.75
1,-3.7,10.75
2,-3.0,12.50
3,-1.3,16.75
4,-0.5,18.75
...,...,...
75,9.7,44.25
76,10.3,45.75
77,10.6,46.50
78,12.1,50.25


# 적합

In [72]:
# step1 
X = df_train[['temp']]
X = np.array(X).astype(np.float32)
y = df_train['sales']
y = np.array(y).astype(np.float32)


# 수동구현

In [85]:
N=2
## 코드1 
predictr = xgb.XGBRegressor(
    n_estimators=N,         # 부스팅 라운드 수
    learning_rate=0.1,         # 학습률
    max_depth=6,               # 최대 깊이
    min_child_weight=1,        # 최소 자식 가중치
    subsample=1.0              # 데이터 샘플링 비율
)
predictr.fit(X,y)
yhat = predictr.predict(X) 

## 코드2 
yhat2 = np.full(y.shape, np.mean(y), dtype=np.float32)
num_boosting_rounds = N
dtrain = xgb.DMatrix(X, label=y)
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 1.0,
}
for i in range(num_boosting_rounds):
    residuals = y - yhat2
    dtrain.set_label(residuals)
    model = xgb.train(params, dtrain, num_boost_round=1)
    new_pred = model.predict(dtrain)
    yhat2 += new_pred

## 코드1,2 비교
diff = ((yhat - yhat2) ** 2).sum()
print(diff)

0.011737638


In [86]:
yhat

array([28.616182, 28.616182, 28.616182, 28.616182, 28.616182, 28.616182,
       29.738379, 29.738379, 29.738379, 29.738379, 29.738379, 29.738379,
       29.738379, 29.738379, 29.738379, 30.033003, 30.033003, 30.033003,
       30.033003, 30.368921, 30.368921, 30.368921, 30.368921, 30.368921,
       30.368921, 30.368921, 30.368921, 30.368921, 30.368921, 30.721207,
       30.974344, 30.974344, 30.974344, 30.974344, 30.974344, 31.278406,
       31.334227, 31.334227, 31.334227, 31.375904, 31.53547 , 31.53547 ,
       31.53547 , 31.53547 , 31.53547 , 31.53547 , 31.784866, 32.072872,
       32.072872, 32.072872, 32.072872, 32.072872, 32.072872, 32.27553 ,
       32.27553 , 32.27553 , 32.27553 , 32.27553 , 32.27553 , 32.27553 ,
       32.27553 , 32.27553 , 32.27553 , 32.53609 , 33.17883 , 33.17883 ,
       33.17883 , 33.17883 , 33.17883 , 33.658257, 33.658257, 33.658257,
       33.658257, 33.658257, 33.658257, 33.658257, 33.658257, 33.658257,
       33.658257, 33.658257], dtype=float32)

In [87]:
yhat2

array([28.60397 , 28.60397 , 28.60397 , 28.60397 , 28.60397 , 28.60397 ,
       29.726261, 29.726261, 29.726261, 29.726261, 29.726261, 29.726261,
       29.726261, 29.726261, 29.726261, 30.020887, 30.020887, 30.020887,
       30.020887, 30.356789, 30.356789, 30.356789, 30.356789, 30.356789,
       30.356789, 30.356789, 30.356789, 30.356789, 30.356789, 30.709076,
       30.962101, 30.962101, 30.962101, 30.962101, 30.962101, 31.265715,
       31.317732, 31.328823, 31.328823, 31.363216, 31.523256, 31.523256,
       31.523256, 31.523256, 31.523256, 31.523256, 31.772177, 32.060776,
       32.060776, 32.060776, 32.060776, 32.060776, 32.060776, 32.26343 ,
       32.26343 , 32.26343 , 32.26343 , 32.26343 , 32.26343 , 32.26343 ,
       32.26343 , 32.26343 , 32.26343 , 32.523876, 33.166615, 33.166615,
       33.166615, 33.166615, 33.166615, 33.646122, 33.646122, 33.646122,
       33.646122, 33.646122, 33.646122, 33.646122, 33.646122, 33.646122,
       33.646122, 33.646122], dtype=float32)